# Test ServiceX FE in Jupyter
The Jupyter environment does some funny things when using async as compared to the command line. This notebook does a very simple dummy query to make sure the basics are all working. I would call this an "integrated" test.

In [1]:
import asyncio
import json
import shutil
from unittest import mock

import pandas as pd
import pytest

import ServiceX_fe as fe

Build a set of mocks to deal with the underlying internals and prevent us from having to out to the outside world

In [2]:
def make_minio_file(mocker, fname):
    r = mocker.MagicMock()
    r.object_name = fname
    return r

class ClientSessionMocker:
    def __init__(self, text, status):
        self._text = text
        self.status = status

    async def text(self):
        return self._text

    async def json(self):
        return json.loads(self._text)

    async def __aexit__(self, exc_type, exc, tb):
        pass

    async def __aenter__(self):
        return self
    
class fget_object_good_copy_callable(mock.MagicMock):
    def __call__(self, *args, **kwargs):
        assert len(args) == 3

        shutil.copy('tests\\sample_servicex_output.root',
                    args[2])

def files_back_1(mocker):
    mocker.patch('minio.api.Minio.list_objects', return_value=[make_minio_file(mocker, 'root:::dcache-atlas-xrootd-wan.desy.de:1094::pnfs:desy.de:atlas:dq2:atlaslocalgroupdisk:rucio:mc15_13TeV:8a:f1:DAOD_STDM3.05630052._000001.pool.root.198fbd841d0a28cb0d9dfa6340c890273-1.part.minio')])
    mocker.patch('minio.api.Minio.fget_object', new_callable=fget_object_good_copy_callable)

def good_transform_request(mocker):
    '''
    Setup to run a good transform request that returns a single file.
    '''
    r1 = ClientSessionMocker(json.dumps({"request_id": "1234-4433-111-34-22-444"}), 200)
    mocker.patch('aiohttp.ClientSession.post', return_value=r1)
    r2 = ClientSessionMocker(json.dumps({"files-remaining": "0", "files-processed": "1"}), 200)
    mocker.patch('aiohttp.ClientSession.get', return_value=r2)

def time_is_short(mocker):
    class AsyncMock(mock.MagicMock):
        async def __call__(self, *args, **kwargs):
            return super(AsyncMock, self).__call__(*args, **kwargs)

    mocker.patch('asyncio.sleep', new_callable=AsyncMock)

## Getting a simple dataset with no async stuff.

Setup the mocks

In [3]:
files_back_1(mock)
time_is_short(mock)
good_transform_request(mock)

Run the test

In [4]:
#def test_good_run_single_ds_1file_noasync_with_loop(good_transform_request, time_is_short, files_back_1):

r = fe.get_data('(valid qastle string)', 'one_ds')
assert isinstance(r, pd.DataFrame)
assert len(r) == 283458

RuntimeError: This event loop is already running